In [17]:
#importing necessary libraries
import pandas as pd
import numpy as np
#wikiedia link
wiki_link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
#fetching data from wiki page
datafrm = pd.read_html(wiki_link,flavor='bs4')
#reading data into dataframe
df= datafrm[0]
df.head()
#making first row of dataframe as header
new_header = df.iloc[0] 
df = df[1:]
df.columns = new_header 
#removing rows where Borough is not assigned
df1 = df[df.loc[:,'Borough'].values !="Not assigned" ]

#resetting index of dataframe
df2 = df1.reset_index(drop=True)

#updating value of neighbourhood with that of value of Borough where value of neighbourhood is not assigned
df2.loc[df2['Neighbourhood'] == "Not assigned", ['Neighbourhood']] = df2.loc[:,'Borough'].values[0]

#combining neighbourhood with same postcode, storing them as comma seprated values
aggregation_functions = {'Postcode': 'first', 'Borough': 'first', 'Neighbourhood': 'unique'}
df_new = df2.groupby(df2['Postcode']).aggregate(aggregation_functions)

df_new2 = df_new.reset_index(drop=True)

#reading csv file into dataframe
df_csv = pd.read_csv("https://cocl.us/Geospatial_data")

#renaming columns
df_csv.rename(columns={'Postal Code': 'Postcode	'}, inplace=True)

#Concatanating dataframes to get required dataframe 
concat = pd.concat([df_new2, df_csv], axis=1)
concat.head()


,Borough,Neighbourhood,Postcode,Postcode,Latitude,Longitude
0,Scarborough,"[Rouge, Malvern]",M1B,M1B,43.806686,-79.194353
1,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",M1C,M1C,43.784535,-79.160497
2,Scarborough,"[Guildwood, Morningside, West Hill]",M1E,M1E,43.763573,-79.188711
3,Scarborough,[Woburn],M1G,M1G,43.770992,-79.216917
4,Scarborough,[Cedarbrae],M1H,M1H,43.773136,-79.239476


In [3]:
import requests # library to handle requests
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.0 MB

The following NEW packages will

In [21]:

venues_map = folium.Map(location=[43.70011, -79.4163], zoom_start=11) # generate map centred around Toronto
# add postalcode as blue circle markers
for Latitude, Longitude in zip(concat.Latitude, concat.Longitude):
    folium.features.CircleMarker(
        [Latitude, Longitude],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map